In [1]:
from sklearn.compose import ColumnTransformer
from optuna import Trial
import pandas as pd
from utils.utils import load_config
import sklearn.ensemble as ens
from utils.PCA_Transformer import PCA_Transform
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler#, OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split

/home/piotr/projects/ml-project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = load_config()
df = pd.read_csv('processed/apartments.csv', index_col = 0)
target = config['sources']['apartments']['target']
pca_columns = config['sources']['apartments']['pca']['columns']
drop_columns = ['rooms']
to_drop = drop_columns + [target] #pca_columns + 
X = df.drop(to_drop, axis =1)
#X_ohe = pd.get_dummies(X)
y = df[target]
cat_cols = ['city']
num_cols = ['squareMeters','centreDistance','buildYear','latitude','longitude']

In [3]:
cols_pca = ['schoolDistance', 'clinicDistance', 'postOfficeDistance', 'kindergartenDistance','restaurantDistance','collegeDistance','pharmacyDistance']

In [4]:
X.head()

,city,type,squareMeters,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,...,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,ownership,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom
0,szczecin,blockOfFlats,63.00,4.0,10.0,1980.0,53.378933,14.625296,6.53,9.0,...,0.105,1.652,NaN,0.413,condominium,yes,yes,yes,no,yes
1,szczecin,blockOfFlats,36.00,8.0,10.0,NaN,53.442692,14.559690,2.15,16.0,...,0.291,0.348,1.404,0.205,cooperative,no,yes,yes,no,yes
2,szczecin,tenement,73.02,2.0,3.0,NaN,53.452222,14.553333,3.24,9.0,...,0.246,0.300,1.857,0.280,condominium,no,no,no,no,no
3,szczecin,tenement,87.60,2.0,3.0,NaN,53.435100,14.532900,2.27,32.0,...,0.359,0.101,0.310,0.087,condominium,yes,yes,no,no,yes
4,szczecin,blockOfFlats,66.00,1.0,3.0,NaN,53.410278,14.503611,4.07,1.0,...,0.704,0.501,2.138,0.514,condominium,no,no,no,no,no


In [5]:
df.isna().sum(  
).sort_values(ascending=False)/len(df) * 100

floor                   18.185665
buildYear               17.302301
collegeDistance          2.988627
floorCount               1.385877
clinicDistance           0.465485
restaurantDistance       0.306797
pharmacyDistance         0.142819
postOfficeDistance       0.132240
schoolDistance           0.074054
kindergartenDistance     0.068765
hasStorageRoom           0.000000
hasSecurity              0.000000
hasElevator              0.000000
hasBalcony               0.000000
hasParkingSpace          0.000000
ownership                0.000000
city                     0.000000
type                     0.000000
poiCount                 0.000000
centreDistance           0.000000
longitude                0.000000
latitude                 0.000000
rooms                    0.000000
squareMeters             0.000000
price                    0.000000
dtype: float64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18905 entries, 0 to 18904
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   city                  18905 non-null  object 
 1   type                  18905 non-null  object 
 2   squareMeters          18905 non-null  float64
 3   rooms                 18905 non-null  float64
 4   floor                 15467 non-null  float64
 5   floorCount            18643 non-null  float64
 6   buildYear             15634 non-null  float64
 7   latitude              18905 non-null  float64
 8   longitude             18905 non-null  float64
 9   centreDistance        18905 non-null  float64
 10  poiCount              18905 non-null  float64
 11  schoolDistance        18891 non-null  float64
 12  clinicDistance        18817 non-null  float64
 13  postOfficeDistance    18880 non-null  float64
 14  kindergartenDistance  18892 non-null  float64
 15  restaurantDistance    18

In [7]:
num_columns = list(X.select_dtypes("number").columns.values)
obj_columns = list(X.select_dtypes("object").columns.values)

In [8]:
num_columns

['squareMeters',
 'floor',
 'floorCount',
 'buildYear',
 'latitude',
 'longitude',
 'centreDistance',
 'poiCount',
 'schoolDistance',
 'clinicDistance',
 'postOfficeDistance',
 'kindergartenDistance',
 'restaurantDistance',
 'collegeDistance',
 'pharmacyDistance']

In [9]:
len(num_columns),len(obj_columns), len(df.columns)

(15, 8, 25)

In [10]:
X[X.select_dtypes("object").columns.to_list()]  = X[X.select_dtypes("object").columns.to_list()].astype('category')
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18905 entries, 0 to 18904
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   city                  18905 non-null  category
 1   type                  18905 non-null  category
 2   squareMeters          18905 non-null  float64 
 3   floor                 15467 non-null  float64 
 4   floorCount            18643 non-null  float64 
 5   buildYear             15634 non-null  float64 
 6   latitude              18905 non-null  float64 
 7   longitude             18905 non-null  float64 
 8   centreDistance        18905 non-null  float64 
 9   poiCount              18905 non-null  float64 
 10  schoolDistance        18891 non-null  float64 
 11  clinicDistance        18817 non-null  float64 
 12  postOfficeDistance    18880 non-null  float64 
 13  kindergartenDistance  18892 non-null  float64 
 14  restaurantDistance    18847 non-null  float64 
 15  college

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=12)

In [42]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression

def instantiate_random_forest(trial : Trial) -> RandomForestRegressor:
  params = {
    'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
    'max_depth': trial.suggest_int('max_depth', 1, 20),
    'max_features': trial.suggest_float('max_features', 0, 1),
    'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
    'n_jobs': -1,
    'random_state': 42
  }
  return RandomForestRegressor(**params)

def instantiate_extra_forest(trial : Trial) -> ExtraTreesRegressor:
  params = {
    'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
    'max_depth': trial.suggest_int('max_depth', 1, 20),
    'max_features': trial.suggest_float('max_features', 0, 1),
    'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
    'n_jobs': -1,
    'random_state': 42
  }
  return ExtraTreesRegressor(**params)

def instantiate_linear_regression(trial : Trial) -> LinearRegression:
  params = {
    'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
    'n_jobs': -1
  }
  return LinearRegression(**params)

Classifier = (
  RandomForestRegressor |
  ExtraTreesRegressor |
  LinearRegression
)

def instantiate_learner(trial : Trial) -> Classifier:
  algorithm = trial.suggest_categorical(
    'algorithm', ['linear', 'forest', 'extra_forest']
  )
  if algorithm=='linear':
    model = instantiate_linear_regression(trial)
  elif algorithm=='forest':
    model = instantiate_random_forest(trial)
  elif algorithm=='extra_forest':
    model = instantiate_extra_forest(trial)
  
  return model

In [43]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from category_encoders import WOEEncoder

Encoder = (
  OrdinalEncoder |
  OneHotEncoder |
  WOEEncoder
)
def instantiate_woe_encoder(trial : Trial) -> WOEEncoder:
  params = {
    'sigma': trial.suggest_float('sigma', 0.001, 5),
    'regularization': trial.suggest_float('regularization', 0, 5),
    'randomized': trial.suggest_categorical('randomized', [True, False])
  }
  return WOEEncoder(**params)

def instantiate_ohe_encoder(trial : Trial) -> OneHotEncoder:
  return OneHotEncoder(handle_unknown='error')

def instantiate_ordinal_encoder(trial : Trial) -> OneHotEncoder:
  return OrdinalEncoder(handle_unknown='error')

def instantiate_encoder(trial : Trial) -> Encoder:
  method = trial.suggest_categorical(
    'encoding_method', ['ordinal', 'onehot', 'woe']
  )
  
  if method=='ordinal':
    encoder = instantiate_ordinal_encoder(trial)
  elif method=='onehot':
    encoder = instantiate_ohe_encoder(trial)
  elif method=='woe':
    encoder = instantiate_woe_encoder(trial)
  
  return encoder

In [74]:
def choose_columns(all_columns : list[str]):
    from utils.utils import load_config
    config = load_config()
    include_columns = config['sources']['apartments']['pca']['columns']
    exclude_columns = []
    if len(set(include_columns).intersection(exclude_columns))>0:
        raise Exception("Column name cant be in included and excluded at the same time")
    
    remain_columns = list(set(all_columns).difference(set(include_columns).union(set(exclude_columns))))
    exclude_columns = set(exclude_columns).intersection(all_columns)
    include_columns = set(include_columns).intersection(all_columns)
    return remain_columns, include_columns, exclude_columns

In [77]:
def initialize_columns(trial : Trial, columns : list[str]) -> list[str]:

  remain_columns, include_columns, exclude_columns = choose_columns(columns)

  choose = lambda column: trial.suggest_categorical(column, [True, False])
  choose_true = lambda column: trial.suggest_categorical(column, [True])
  choose_false = lambda column: trial.suggest_categorical(column, [False])
  choices = [*filter(choose, remain_columns)]
  choices_true =  [*filter(choose_true, include_columns)]
  choices_false =  [*filter(choose_false, exclude_columns)]
  print(f"choices: {choices}, choices_true: {choices_true},  choices_false: {choices_false}")
  return choices + choices_true

In [13]:
from sklearn.preprocessing import RobustScaler
#RobustScaler better than StandardScaler because of outliers
def instantiate_robust_scaler(trial : Trial) -> RobustScaler:
  params = {
    'with_centering': trial.suggest_categorical(
      'with_centering', [True, False]
    ),
    'with_scaling': trial.suggest_categorical(
      'with_scaling', [True, False]
    )
  }
  return RobustScaler(**params)

In [66]:
from sklearn.pipeline import Pipeline


def instantiate_numerical_pipeline(trial : Trial, columns: list[str] = None) -> Pipeline:
  pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', instantiate_robust_scaler(trial)),
    ('pca', PCA_Transform(columns))
  ])
  return pipeline

def instantiate_categorical_pipeline(trial : Trial, columns: list[str] = None) -> Pipeline:
  pipeline = Pipeline([
    ('encoder', instantiate_encoder(trial)),
    #('imputer', KNNImputer(n_neighbors=5)), categorical nans were replaced with 'Other'?
    ('scaler', instantiate_robust_scaler(trial)) 
  ])
  return pipeline

In [80]:
from sklearn.compose import ColumnTransformer

def instantiate_processor(trial : Trial, numerical_columns : list[str], categorical_columns : list[str]) -> ColumnTransformer:

  selected_numerical_columns = initialize_columns(trial, numerical_columns)
  selected_categorical_columns = initialize_columns(trial, categorical_columns)

  numerical_pipeline = instantiate_numerical_pipeline(trial, selected_numerical_columns)
  categorical_pipeline = instantiate_categorical_pipeline(trial)
  
  

  processor = ColumnTransformer([
   ('numerical_pipeline', numerical_pipeline, selected_numerical_columns),
   ('categorical_pipeline', categorical_pipeline, selected_categorical_columns)
  ])
  return processor

def instantiate_model(trial : Trial, numerical_columns : list[str], categorical_columns : list[str]) -> Pipeline:
  processor = instantiate_processor(
    trial, numerical_columns, categorical_columns
  )
  learner = instantiate_learner(trial)
  
  model = Pipeline([
    ('processor', processor),
    ('learner', learner)
  ])
  return model

In [68]:
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from pandas import DataFrame, Series
import numpy as np

def objective(trial : Trial, X : DataFrame, y : np.ndarray | Series, numerical_columns : Optional[list[str]]=None, categorical_columns : Optional[list[str]]=None, random_state : int=42) -> float:
  
  if numerical_columns is None:
    numerical_columns = [
      *X.select_dtypes(exclude=['object', 'category']).columns
    ]
  
  if categorical_columns is None:
    categorical_columns = [
      *X.select_dtypes(include=['object', 'category']).columns
    ]
  print(categorical_columns, X.columns)
  model = instantiate_model(trial, numerical_columns, categorical_columns)
  kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
  mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False, needs_proba=False)
  scores = cross_val_score(model, X, y, scoring=mape_scorer, cv=kf)
  return np.min([np.mean(scores), np.median([scores])])

In [22]:
cat_columns = list(X.select_dtypes("category").columns.values)
cat_columns

['city',
 'type',
 'ownership',
 'hasParkingSpace',
 'hasBalcony',
 'hasElevator',
 'hasSecurity',
 'hasStorageRoom']

In [79]:
from optuna import create_study

study = create_study(study_name='optimization', direction='maximize')

study.optimize(lambda trial: objective(trial, X_train, y_train,numerical_columns=num_columns, categorical_columns=cat_columns), n_trials=1)

[I 2024-02-12 12:05:43,882] A new study created in memory with name: optimization
[W 2024-02-12 12:05:43,883] Trial 0 failed with parameters: {} because of the following error: TypeError('choose_columns() takes 1 positional argument but 2 were given').
Traceback (most recent call last):
  File "/home/piotr/projects/ml-project/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_115255/2676147570.py", line 5, in <lambda>
    study.optimize(lambda trial: objective(trial, X_train, y_train,numerical_columns=num_columns, categorical_columns=cat_columns), n_trials=1)
  File "/tmp/ipykernel_115255/142445651.py", line 19, in objective
    model = instantiate_model(trial, numerical_columns, categorical_columns)
  File "/tmp/ipykernel_115255/1493850561.py", line 20, in instantiate_model
    processor = instantiate_processor(
  File "/tmp/ipykernel_115255/1493850561.py", line 5, in instantiate_processor
   

['city', 'type', 'ownership', 'hasParkingSpace', 'hasBalcony', 'hasElevator', 'hasSecurity', 'hasStorageRoom'] Index(['city', 'type', 'squareMeters', 'floor', 'floorCount', 'buildYear',
       'latitude', 'longitude', 'centreDistance', 'poiCount', 'schoolDistance',
       'clinicDistance', 'postOfficeDistance', 'kindergartenDistance',
       'restaurantDistance', 'collegeDistance', 'pharmacyDistance',
       'ownership', 'hasParkingSpace', 'hasBalcony', 'hasElevator',
       'hasSecurity', 'hasStorageRoom'],
      dtype='object')


TypeError: choose_columns() takes 1 positional argument but 2 were given

In [106]:
study.

-0.12302513820548737

In [85]:
freq = {'a':9,'b':3,"c":12,"d":1}
freq_s = list(freq.keys())

In [86]:
freq_s.sort(key = freq.get)

In [87]:
freq_s

['d', 'b', 'a', 'c']